![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


# The output

Your query should return a table in the following format:
| industry  | year | num\_unicorns       | average\_valuation\_billions |
| --------- | ---- | ------------------- | ---------------------------- |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |
| industry1 | 2021 |        ---          |             ---              |
| industry2 | 2020 |        ---          |             ---              |
| industry3 | 2019 |        ---          |             ---              |

Where `industry1`, `industry2`, and `industry3` are the three top-performing industries.

In [38]:
SELECT 
i.industry, 
COUNT(*)
FROM industries AS i
JOIN dates AS d
ON i.company_id = d.company_id
AND EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
GROUP BY i.industry
ORDER BY COUNT(*) DESC
LIMIT 3;

,industry,count
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


In [39]:
WITH cte AS(SELECT 
	i.industry, 
	COUNT(*)
	FROM industries AS i
	JOIN dates AS d
	ON i.company_id = d.company_id
	AND EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
	GROUP BY i.industry
	ORDER BY COUNT(*) DESC
	LIMIT 3)
SELECT 
i.industry,
EXTRACT(YEAR FROM d.date_joined) AS year,
COUNT(*) AS num_unicorns,
ROUND(AVG(f.valuation)::numeric/100000000000,2) AS average_valuation_billions
FROM public.industries AS i
JOIN public.dates AS d
ON i.company_id = d.company_id
AND EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
AND i.industry IN (SELECT industry FROM cte)
JOIN public.funding AS f
ON i.company_id = f.company_id
GROUP BY i.industry,EXTRACT(YEAR FROM d.date_joined);


,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2019,12,0.03
1,E-commerce & direct-to-consumer,2020,16,0.04
2,E-commerce & direct-to-consumer,2021,47,0.02
3,Fintech,2019,20,0.07
4,Fintech,2020,15,0.04
5,Fintech,2021,138,0.03
6,Internet software & services,2019,13,0.04
7,Internet software & services,2020,20,0.04
8,Internet software & services,2021,119,0.02


In [40]:
WITH cte1 AS(
	SELECT 
	i.industry, 
	COUNT(i.*)
	FROM industries AS i
	JOIN dates AS d
	ON i.company_id = d.company_id
	AND EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
	GROUP BY i.industry
	ORDER BY COUNT(*) DESC
	LIMIT 3
),
cte2 AS (
	SELECT 
	i.industry,
	EXTRACT(YEAR FROM d.date_joined) AS year,
	COUNT(i.*) AS num_unicorns,
	ROUND(AVG(f.valuation)::numeric/1000000000,2) AS average_valuation_billions
	FROM public.industries AS i
	JOIN public.dates AS d
	ON i.company_id = d.company_id
	AND EXTRACT(YEAR FROM d.date_joined) IN ('2019', '2020', '2021')
	AND i.industry IN (SELECT industry FROM cte1)
	JOIN public.funding AS f
	ON i.company_id = f.company_id
	GROUP BY i.industry,EXTRACT(YEAR FROM d.date_joined)
)
SELECT
* 
FROM cte2
ORDER BY YEAR DESC, num_unicorns DESC;


,industry,year,num_unicorns,average_valuation_billions
0,Fintech,2021,138,2.75
1,Internet software & services,2021,119,2.15
2,E-commerce & direct-to-consumer,2021,47,2.47
3,Internet software & services,2020,20,4.35
4,E-commerce & direct-to-consumer,2020,16,4.00
5,Fintech,2020,15,4.33
6,Fintech,2019,20,6.80
7,Internet software & services,2019,13,4.23
8,E-commerce & direct-to-consumer,2019,12,2.58
